In [3]:
import sys
sys.path.append('..')
from rosiellm import RosieLLM
from rosiellm.RosieSSH import RosieSSH, RosieAuth
import base64
import time

### Test RosieSSH

In [ ]:
rosie_ssh = RosieSSH()
rosie_ssh.connect()

In [ ]:
print("Streaming: ", "\n", "---------")
output = rosie_ssh.execute_command('squeue', streaming=True)
print("---------", "\n", "End Streaming")
print("Output:", "\n", output)

In [ ]:
commands = ['hostname', 'hostname', 'uptime', 'uptime', 'echo "Hello World!"', 'echo "Hello World!"']
for command in commands:
    #track time
    start = time.time()
    output = rosie_ssh.execute_command(command)
    print(f"Command[{command}, time: {time.time() - start:.2f}]:", end="")
    print(output, end="\n")
    # time.sleep(1)

In [ ]:
rosie_ssh.close()

#### Test RosieAuth

In [ ]:
credentials = "username:password"
encoded_credentials = base64.b64encode(credentials.encode("utf-8")).decode("utf-8")
authorization_header = f"Basic {encoded_credentials}"

rosie_auth = RosieAuth("username", "password")
rosie_auth_header = f"Basic {rosie_auth.get_rosie_auth()}"

print("Src Authorization Header:", f'"{authorization_header}"')
print("Lib Authorization Header:", f'"{rosie_auth_header}"')

### Test vLLM Server

In [6]:
client = RosieLLM(
    rosie_username="milleraa",
    partition="teaching",
    gpus=2,
    # log_level="debug"
)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:rosiellm.RosieJob:Updated partition from teaching > teaching
INFO:rosiellm.RosieJob:Updated gpus from 2 > 2
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 1] sftp session closed.



Submitted batch job 178119


INFO:rosiellm.RosieJob:Job URL Found
INFO:rosiellm.RosieLLM:See your job progress here:
INFO:rosiellm.RosieLLM:https://dh-ood.hpc.msoe.edu/pun/sys/dashboard/files/fs//data/ai_club/RosieLLM/out/milleraa_out.txt


Job has been launched, waiting for server to start (this can take over a minute)...


In [ ]:
# call __dict__ recursively to get all attributes
d = client.__dict__
for key in d:
    # if key is a class object, call __dict__ recursively
    if hasattr(d[key], '__dict__'):
        print(f"{key}:")
        for subkey in d[key].__dict__:
            if (subkey == "_custom_headers"):
                # hide password to avoid accidental exposure
                header_dict = d[key].__dict__[subkey]
                header_str = str(header_dict)
                idx1 = header_str.find("'Basic ")
                idx2 = header_str.find("'", idx1 + 1)
                if idx1 != -1 and idx2 != -1:
                    print(f'    {subkey}: {header_str[:idx1 + len("Basic ")]} <***>{header_str[idx2:]}')
                else:
                    print(f'    {subkey}: {header_str}')
            else:
                print(f"    {subkey}: {d[key].__dict__[subkey]}")
    else:
        print(f"{key}: {d[key]}")

In [ ]:
# client = client.http_client

In [ ]:
client.isRunning

In [8]:
completion = client.chat.completions.create(
  model=client.model,
  messages=[
    {"role": "user", "content": "What is AI Club at MSOE?"},
  ],
  stream=True,
)

for chunk in completion:
  print(chunk.choices[0].delta.content, end="")
# print(completion.choices[0].message.content)

INFO:rosiellm.RosieLLM:Checking server health...
INFO:rosiellm.RosieLLM:Health check status: 200: ok
INFO:rosiellm.RosieLLM:Server is running.
INFO:httpx:HTTP Request: POST https://dh-ood.hpc.msoe.edu/node/dh-node18.hpc.msoe.edu/1234/v1/chat/completions "HTTP/1.1 200 OK"


The AI Club at Milwaukee School of Engineering (MSOE) is a student-run organization that focuses on artificial intelligence (AI) and its applications. The club provides a platform for students, faculty, and industry professionals to come together, share knowledge, and work on AI-related projects.

The AI Club at MSOE aims to:

1. Educate members about AI concepts, techniques, and applications through workshops, seminars, and guest lectures.
2. Provide opportunities for members to work on AI-related projects, such as machine learning, natural language processing, computer vision, and robotics.
3. Foster a community of like-minded individuals who are interested in AI and its potential to transform industries and society.
4. Organize hackathons, competitions, and events to challenge members to develop innovative AI solutions.
5. Facilitate connections between MSOE students and industry professionals in the AI field, potentially leading to internships, job opportunities, and collaborations

ERROR:paramiko.transport:Socket exception: An existing connection was forcibly closed by the remote host (10054)
ERROR:paramiko.transport:Socket exception: An existing connection was forcibly closed by the remote host (10054)
